### Задача 1 

На вход доступен следующий фрейм данных:  

| id | timestamp |
|---|---|
| 1 | 1562007679 |
| 1 | 1562007710 |
| 1 | 1562007720 |
| 1 | 1562007750 |
| 2 | 1564682430 |
| 2 | 1564682450 |
| 2 | 1564682480 |

`id` — уникальный идентификатор пользователя  
`timestamp` — время действия на сайте, совершенное данным пользователем

Для каждого id рассчитайте усредненную длину сессии в рамках суток. Усредненная длина сессии рассчитывается как разница между первым и последним действием, произведенным на сайте.

In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\User\\anaconda3\\lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *

spark = SparkSession.builder.master("local[8]")\
    .appName("SparkFirst")\
    .config("spark.executor.memory", "10g")\
    .config("spark.executor.cores", 5)\
    .config("spark.dynamicAllocation.enabled", "true")\
    .config("spark.dynamicAllocation.maxExecutors", 5)\
    .config("spark.shuffle.service.enabled", "true")\
    .getOrCreate()

In [3]:
import pandas as pd

data_schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('timestamp', IntegerType(), True)
])

data = [(1, 1562007679), (1, 1562007710), (1, 1562007720), (1, 1562007750), (2, 1564682430), (2, 1564682450), (2, 1564682480)]
rdd = spark.createDataFrame(data = data, schema = data_schema)
rdd.createOrReplaceTempView("sales_df")
rdd.show()

+---+----------+
| id| timestamp|
+---+----------+
|  1|1562007679|
|  1|1562007710|
|  1|1562007720|
|  1|1562007750|
|  2|1564682430|
|  2|1564682450|
|  2|1564682480|
+---+----------+



In [7]:
recency = spark.sql("""
WITH users_session AS (
    select 
        id,
        date(cast(timestamp AS timestamp)) as date,
        min(cast(timestamp AS timestamp)) AS d_min,
        max(cast(timestamp AS timestamp)) AS d_max
    from sales_df
    group by id, date
),
users_session_duration AS (
    select 
        id,
        date,
        TIMESTAMPDIFF(SECOND,  d_min, d_max) as session_duration
    from users_session
),
users_session_duration_converted AS (
    select
        id,
        date,
        DATE_FORMAT(TIMESTAMPADD(SECOND, session_duration, '2000-01-01 00:00:00'), "HH:mm:ss") AS session_time
    from users_session_duration
)

select 
*
from users_session_duration_converted
""")
recency.show()

+---+----------+--------------+
| id|      date|converted_time|
+---+----------+--------------+
|  1|2019-07-01|      00:01:11|
|  2|2019-08-01|      00:00:50|
+---+----------+--------------+



In [ ]:
spark.stop()